In [1]:
with open('/content/article1.txt') as f:
    article_text = f.read()

In [2]:
#import module
import re
import nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
article_text = article_text.lower()
article_text

'noor hisham: several reasons for increase in education clusters.\nhttps://www.nst.com.my/news/nation/2022/01/766418/noor-hisham-several-reasons-increase-education-clusters\nkuala lumpur: the reasons for the exponential increase of covid-19 education clusters include failure to adhere to the standard operating procedures (sop) and disregarding quarantine rules.\n\nhealth director-general tan sri dr noor hisham abdullah said these were identified by the various district health offices in which education clusters and cases had appeared.\n\nhe said among other reasons were individuals who were symptomatic but still attended classes and detected late; crowded spaces that were not cleaned and disinfected periodically; and, poor ventilation systems in classes and hostels.\n\nhe revealed that 107 clusters in the education category had been reported between jan 1 and 25, involving 4,633 cases.\ndr noor hisham said of these reported cases, 4,092 were fully vaccinated and 112 had received the bo

In [4]:
#data preprocessing

In [5]:
# remove spaces, punctuations and numbers
clean_text = re.sub('[^a-zA-Z]', ' ', article_text)
clean_text = re.sub('\s+', ' ', clean_text)
clean_text

'noor hisham several reasons for increase in education clusters https www nst com my news nation noor hisham several reasons increase education clusters kuala lumpur the reasons for the exponential increase of covid education clusters include failure to adhere to the standard operating procedures sop and disregarding quarantine rules health director general tan sri dr noor hisham abdullah said these were identified by the various district health offices in which education clusters and cases had appeared he said among other reasons were individuals who were symptomatic but still attended classes and detected late crowded spaces that were not cleaned and disinfected periodically and poor ventilation systems in classes and hostels he revealed that clusters in the education category had been reported between jan and involving cases dr noor hisham said of these reported cases were fully vaccinated and had received the booster shots the majority of the cases or per cent were under category a

In [6]:
# split into sentence list
sentence_list = nltk.sent_tokenize(article_text)
sentence_list

['noor hisham: several reasons for increase in education clusters.',
 'https://www.nst.com.my/news/nation/2022/01/766418/noor-hisham-several-reasons-increase-education-clusters\nkuala lumpur: the reasons for the exponential increase of covid-19 education clusters include failure to adhere to the standard operating procedures (sop) and disregarding quarantine rules.',
 'health director-general tan sri dr noor hisham abdullah said these were identified by the various district health offices in which education clusters and cases had appeared.',
 'he said among other reasons were individuals who were symptomatic but still attended classes and detected late; crowded spaces that were not cleaned and disinfected periodically; and, poor ventilation systems in classes and hostels.',
 'he revealed that 107 clusters in the education category had been reported between jan 1 and 25, involving 4,633 cases.',
 'dr noor hisham said of these reported cases, 4,092 were fully vaccinated and 112 had recei

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Word Frequencies

In [9]:
stopwords = nltk.corpus.stopwords.words('english')
#create dictionary
word_frequencies = {}
for word in nltk.word_tokenize(clean_text):
    if word not in stopwords:
        if word not in word_frequencies:
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

In [10]:
maximum_frequency = max(word_frequencies.values())

for word in word_frequencies:
    word_frequencies[word] = word_frequencies[word] / maximum_frequency

In [11]:
#Calculate sentence Scores

In [12]:
sentence_scores = {}

for sentence in sentence_list:
    for word in nltk.word_tokenize(sentence):
        if word in word_frequencies and len(sentence.split(' ')) < 30:
            if sentence not in sentence_scores:
                sentence_scores[sentence] = word_frequencies[word]
            else:
                sentence_scores[sentence] += word_frequencies[word]

In [13]:
word_frequencies

{'abdullah': 0.07692307692307693,
 'adhere': 0.07692307692307693,
 'also': 0.07692307692307693,
 'among': 0.07692307692307693,
 'appeared': 0.07692307692307693,
 'assessment': 0.07692307692307693,
 'asymptomatic': 0.07692307692307693,
 'attended': 0.07692307692307693,
 'booster': 0.07692307692307693,
 'cases': 0.6153846153846154,
 'categories': 0.07692307692307693,
 'category': 0.15384615384615385,
 'cent': 0.46153846153846156,
 'classes': 0.23076923076923078,
 'cleaned': 0.07692307692307693,
 'close': 0.15384615384615385,
 'closed': 0.07692307692307693,
 'clusters': 0.6923076923076923,
 'com': 0.07692307692307693,
 'compulsory': 0.07692307692307693,
 'conducting': 0.07692307692307693,
 'contact': 0.15384615384615385,
 'council': 0.07692307692307693,
 'covid': 0.23076923076923078,
 'crowded': 0.07692307692307693,
 'department': 0.15384615384615385,
 'departments': 0.07692307692307693,
 'detected': 0.15384615384615385,
 'development': 0.07692307692307693,
 'director': 0.0769230769230769

In [14]:
sentence_scores

{'"a 313 per cent increase was reported during the 3rd ew.': 1.8461538461538463,
 '"during the 1st epidemiological week (ew), only three educational clusters were detected.': 1.7692307692307692,
 '"the majority of the cases, or 99.7 per cent, were under category 1 (asymptomatic) and 2 (with mild symptoms) without lung infection.': 2.4615384615384617,
 '"the remaining 12 cases were under categories 3 and 4. all of them are stable and now receiving treatment at health facilities," he said in a statement today.': 2.4615384615384617,
 'dr noor hisham said cases were rising following the reopening of school sessions starting jan 9.': 3.8461538461538476,
 'dr noor hisham said of these reported cases, 4,092 were fully vaccinated and 112 had received the booster shots.': 3.6153846153846168,
 'he revealed that 107 clusters in the education category had been reported between jan 1 and 25, involving 4,633 cases.': 3.3076923076923075,
 'health director-general tan sri dr noor hisham abdullah said 

In [15]:
#Summarization

In [22]:
# get top 5 sentences
import heapq
summary = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)
summary=" ".join(summary)
print(summary)

health director-general tan sri dr noor hisham abdullah said these were identified by the various district health offices in which education clusters and cases had appeared. dr noor hisham said cases were rising following the reopening of school sessions starting jan 9. noor hisham: several reasons for increase in education clusters. https://www.nst.com.my/news/nation/2022/01/766418/noor-hisham-several-reasons-increase-education-clusters
kuala lumpur: the reasons for the exponential increase of covid-19 education clusters include failure to adhere to the standard operating procedures (sop) and disregarding quarantine rules. dr noor hisham said of these reported cases, 4,092 were fully vaccinated and 112 had received the booster shots.


In [23]:
outFileName="summary.txt"
outFile=open(outFileName, "w")
outFile.write(summary)
outFile.close()